In [ ]:
# For DL modeling
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn

# For number-crunching
import numpy as np
import scipy.stats as stats

# For dataset management
import pandas as pd
from sklearn.model_selection import train_test_split

# For data visualization
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')
import seaborn as sns

# For timing computations
import time

import copy

import sklearn.metrics as skm

import sys

# For doing PCA on the model output
from sklearn.decomposition import PCA

# Import and process the data

In [ ]:
# Import dataset
mnist_dataset = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'), delimiter=',')

#Extract labels (number IDs) and remove from data
labels = mnist_dataset[:, 0]
data   = mnist_dataset[:, 1:]

# Normalize the data to a range of [0, 1]
data_norm = data / np.max(data)

# Convert to tensor
data_tensor   = torch.tensor(data_norm).float()

# Create the DL model

In [ ]:
# Create a class for the model
def create_the_MNIST_AE():
    """
    AUTOENCODER_ LATENT_CODE
    """
    class ae_net(nn.Module):
        def __init__(self):
            super().__init__()

            # Input layer
            self.input = nn.Linear(784, 150)

            # Encoder layer
            self.enc = nn.Linear(150, 15)

            # Latent layer
            self.lat = nn.Linear(15, 150)

            # Decoder layer
            self.dec = nn.Linear(150, 784)
    
        # Forward pass
        def forward(self, x):
            x     = F.relu(self.input(x))
            codex = F.relu(self.enc(x)) # Output the hidden-layer activation
            x     = F.relu(self.lat(codex))
            y     = torch.sigmoid(self.dec(x))

            return y, codex
    
    # Create the model instance
    net = ae_net()

    # Loss function
    loss_func = nn.MSELoss()

    # Optimizer
    optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

    return net, loss_func, optimizer

In [ ]:
# Test the model with a bit of data
net, loss_func, optimizer = create_the_MNIST_AE()

X     = data_tensor[:5, :]
y_hat = net(X)

print(f'Input shape: {X.shape}', '\n')
print(type(y_hat), len(y_hat), '\n')
print(f'Shape of model output: {y_hat[0].shape}', '\n')
print(f'Shape of encoding layer output: {y_hat[1].shape}', '\n')

# Create a function that trains the model

In [ ]:
def train_the_model():
    """
    AUTOENCODER_DENOISING_MNIST|LATENT_CODE
    """

    num_epochs = 10000

    # Create a new model
    net, loss_func, optimizer = create_the_MNIST_AE()

    # Initialize losses
    losses = torch.zeros(num_epochs)

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Select a random set of images
        random_idx = np.random.choice(data_tensor.shape[0], size=32)
        X          = data_tensor[random_idx, :]

        # Forward pass and loss
        y_hat = net(X)
        loss  = loss_func(y_hat, X)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Losses in this epoch
        losses[epoch_i] = loss.item()
    
    # End epochs
    # Function output
    return losses, net

# Run the model and show the results!

In [ ]:
# Train the model
losses, net = train_the_model()
print(f'Final loss: {losses[-1]:.4f}')

# Visualize the losses
plt.plot(losses, '.-')
plt.xlabel('Epochs')
plt.ylabel('Model loss')
plt.show()

# Inspect the latent "code" of the model

In [ ]:
# Output the latent layer
# Push through the entire dataset
y_hat, latent = net(data_tensor)

# Print sizes
print(f'{y_hat.shape}, {latent.shape}')

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].hist(latent.flatten().detach(), 100)
ax[0].set_xlabel('Latent Activation value')
ax[0].set_ylabel('Count')
ax[0].set_title('Distribution of latent units activation')

ax[1].imshow(latent.detach(), aspect='auto', vmin=0, vmax=10)
ax[1].set_xlabel('Latent node')
ax[1].set_ylabel('Image number')
ax[1].set_title('All latent activations')

plt.show()

In [ ]:
# compute the average latent activation for each digit type

# initialize output matrix (latent shape by 10 digits)
sourcecode = np.zeros((latent.shape[1],10))


# loop over digit categories
for i in range(10):

  # find all pictures of this category
  digidx = np.where(labels==i)

  # average the latent layer output
  sourcecode[:,i] = torch.mean(latent[digidx,:],axis=1).detach()


# let's see what it looks like!
fig = plt.figure(figsize=(8,5))

plt.plot(sourcecode,'s-')
plt.legend(range(10),loc=(1.01,.4))
plt.xticks(range(15))
plt.xlabel('Latent node number')
plt.ylabel('Activation')
plt.title("The model's internal representation of the numbers")
plt.show()

# Explore the reduced-compressed space with PCA

In [ ]:
# compute and fit the PCA
pcaData = PCA(n_components=15).fit(data) # 15 components to match latent, but it's just to speed computation time
pcaCode = PCA(               ).fit(latent.detach())


# plot the eigenspectra (scree plot)
plt.plot(100*pcaData.explained_variance_ratio_,'s-',label='Data PCA')
plt.plot(100*pcaCode.explained_variance_ratio_,'o-',label='Code PCA')
plt.xlabel('Components')
plt.ylabel('Percent variance explained')
plt.title('PCA scree plot')
plt.legend()
plt.show()

In [ ]:
# compute the projection of the data onto the PC axes
scoresData = pcaData.fit_transform(data)
scoresCode = pcaCode.fit_transform(latent.detach())

# plot the data separately per number
fig,ax = plt.subplots(1,2,figsize=(15,5))

for lab in range(10):
  ax[0].plot(scoresData[labels==lab,0],scoresData[labels==lab,1],'o',markersize=3,alpha=.4)
  ax[1].plot(scoresCode[labels==lab,0],scoresCode[labels==lab,1],'o',markersize=3,alpha=.4)

for i in range(2):
  ax[i].set_xlabel('PC1 projection')
  ax[i].set_ylabel('PC2 projection')
  ax[i].legend(range(10))

ax[0].set_title('PCA of data')
ax[1].set_title('PCA of latent code')
plt.show()

In [ ]:
# This cell is not important! It's just the code I used to make the figure in the slide. I decided to leave it here FYI.

fig,ax = plt.subplots(1,3,figsize=(15,3))

ax[0].imshow(dataT[0,:].view(28,28),cmap='gray')

ax[1].plot(dataT[0,:],'ks')
ax[1].set_xlabel('Pixels (vectorized)')
ax[1].set_ylabel('Intensity value')

ax[2].plot(latent[0,:].detach(),'ks')
ax[2].set_xlabel('Latent units')
ax[2].set_ylabel('Activation (a.u.)')

plt.show()

# Additional explorations

In [ ]:
# 1) Are you surprised that the latent activations (e.g., from the histogram) are all non-negative? Is that because of 
#    the image normalization, or what is causing those values to be all non-negative?
# 
# 2) Averages don't tell the whole story. Redraw the "Model's internal representation" line plot but using standard 
#    deviation instead of mean. This graph will tell you if any numbers, or units, have particularly higher variability
#    than others. Is this the case, and does the std plot give you any more insight into the model's learned representation?
# 
# 3) The PC-space plots are tricky to interpret: This is a 15-dimensional space but 13 dimensions are projected onto two.
#    It's possible that the numbers are better separated in other dimensions, just like a 2D photograph of someone standing
#    behind a tree makes them inseparable whereas they are separable in the original 3D space. Modify the plot to show
#    PC dimensions 2&3 instead of 1&2. 
# 